# Elements Of Programming Interviews
## Searching Algorithms
### Track 8: 12.1, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 12.11, 12.12

### 12.1 - Search A Sorted Array For The First Occurence Of K
>Write a method that takes a sorted array and a key and returns the index of the *first* occurence of that key in the array.

First a basic binary search

In [1]:
def binary_search(arr, k):
    s = 0; e = len(arr) - 1
    #s:start_index, e:end_index, m:middle_index
    while s <= e:
        m = int(s + (e - s) / 2)
        if arr[m] < k:
            s = m + 1
        elif arr[m] == k:
            return m
        else:
            e = m - 1
    return -1

In [2]:
binary_search(range(10), 2), binary_search(range(0,10,2), 3)

(2, -1)

I can use a basic binary search, once I find a key, then all the other equivalent keys must be directly adjacent to the curr index. The slower way to find the first index would be to loop backwards until the last occurence of the key is found. A better implementation would be to continue the binary search.

In [3]:
def first_occurence_bf(arr, k):
    s = 0; e = len(arr) - 1
    #s:start_index, e:end_index, m:middle_index
    while s <= e:
        m = int(s + (e - s) / 2)
        if arr[m] < k:
            s = m + 1
        elif arr[m] == k:
            while m > 0 and arr[m - 1] == k:
                m -= 1
            return m
        else:
            e = m - 1
    return -1

In [4]:
first_occurence_bf([0, 2, 2, 2, 2, 2, 2, 3, 4, 5, 6, 7], 2)

1

In [5]:
def first_occurence(arr, k):
    start = 0; end = len(arr) - 1
    #s:start_index, e:end_index, m:middle_index
    looking_for_first = False
    result = -1
    while start <= end:
        mid = int(start + (end - start) / 2)
        if arr[mid] < k:
            start = mid + 1
        elif arr[mid] == k:
            result = mid
            end = mid - 1
        else: #arr[mid] < k
            end = mid - 1
    return result

In [6]:
(first_occurence([1, 2, 3, 3, 3, 3, 3, 5, 6, 7, 8], 3),
 first_occurence([0], 1), first_occurence([1,1,1,1,1,1,1,1,1,1], 1),
 first_occurence([1, 2, 3, 5, 6, 6, 6, 6], 6),
 first_occurence([2]* 500000, 2)
)

(2, -1, 0, 4, 0)

The difference in speed between the two methods is very noticeable

In [7]:
%timeit first_occurence([2] * 1000000, 2)

100 loops, best of 3: 2.47 ms per loop


In [8]:
%timeit first_occurence_bf([2] * 1000000, 2)

10 loops, best of 3: 81.6 ms per loop


### 12.4 - Search A Cyclically Sorted Array
An array is said to be cyclically sorted if it is possible to cyclically shift its entries so that it becomes sorted.
For example:
>[378, 478, 550, 631, 103, 203, 220, 234, 279, 368]

is cyclically sorted--a cyclic left shift by 4 leads to a sorted array.

Design an $O(logn)$ algorithm for finding the position of the samllest element in a cyclically sorted array. Assume all elements are **distinct**. For example, for the array example above, the algorithm should return 4 (the index of 103).

First I want to find the size of the cycles in the array.

In [9]:
def find_smallest_rec(arr):
    if len(arr) == 0:
        return None
    else:
        return smallest_helper(arr, 0, len(arr) - 1)

def smallest_helper_rec(arr, left, right):
    #from IPython.core.debugger import Tracer    
    #Tracer()()
    if right - left == 1:
        return arr[right]
    else:
        mid = int(left + (right - left) / 2)
        if arr[mid] < arr[right]:
            right = mid
        elif arr[mid] > arr[right]:
            left = mid
        return smallest_helper(arr, left, right)

In [10]:
def find_smallest(arr):
    left = 0; right = len(arr) - 1
    while left < right:
        mid = int(left + (right - left) / 2)
        if arr[mid] < arr[right]:
            # min must be in [left : mid]
            right = mid
        elif arr[mid] > arr[right]:
            # min must be in [mid + 1 : right]
            left = mid + 1
    return ("Index:", left, "Val:", arr[left])

In [12]:
find_smallest([5,6,1,2]), find_smallest(list(range(50,100)) + list(range(1, 25)))

(('Index:', 2, 'Val:', 1), ('Index:', 50, 'Val:', 1))

### 12.5 - Compute The Integer Square Root
Write a program which takes a nonnegative integer and returns the largest integer whose square is less than or equal to the given integer. For exaple, 

e.g. if the input is 16, return 4;

e.g. if the input is 300, return 17, since $17^2 = 289$ and $18^2 = 324 > 300$

>obvious brute force solution is iterate through a range, calculating the square for each succesive number

In [22]:
def integer_square_root_bf(num):
    counter = 0
    while True:
        if counter ** 2 > num:
            break
        counter += 1
    return counter - 1

In [27]:
integer_square_root_bf(300), integer_square_root_bf(16)

(17, 4)

>next idea is to double the counter at every loop. Once the counter squared is greater than the passed in number, check the values between the previous counter value and the current counter value.

In [34]:
def find_root_in_range(num, start, end):
    for i in range(start, end+1):
        if i ** 2 > num:
            return i - 1
def integer_square_root_bf_2(num):
    prev = 1; counter = 1
    while True:
        if counter ** 2 > num:
            ret = find_root_in_range(num, prev, counter)
            break
        prev = counter
        counter = counter * 2
    return ret

In [49]:
#sqrt(42353243) == 6507.9369
%timeit integer_square_root_bf(42353243534555)

1 loop, best of 3: 2.29 s per loop


In [51]:
%timeit integer_square_root_bf_2(42353243534555)

1 loop, best of 3: 775 ms per loop


>next approach utilizes a few key insights
* If $x^2 > k$, then all values greater than x are not candidates
* If $x^2 < k$, then all values less than x are not candidates
* the ability to eliminate large amounts of candidates can improve preformance siginifcantly

1. Initialize a range of candidates [0, k]
2. Pick a val  in the middle $[(right - left) / 2]$
    * If $val^2 > k$, change range of candidates to [start, val-1]
    * If $val^2 < k$, change range of candidates to [val+1, end]

In [69]:
def integer_square_root(k):
    candidates = range(0, k + 1)
    left = 0; right = k
    #once the right and left indicies are adjacent, the left will point at the res
    while left <= right:
        middle = int(left + (right - left) / 2)
        #candidates[middle] equal middle
        squared = middle ** 2
        if squared > k:
            right = middle - 1
        else: # squared ** 2 <= k:
            left = middle + 1
    return left - 1

In [70]:
integer_square_root(42353243), integer_square_root(0), integer_square_root(16)

(6507, 0, 4)

The difference in preformance between these 3 approaches

In [ ]:
%timeit integer_square_root_bf(42353243534555)
%timeit integer_square_root_bf_2(42353243534555)
%timeit integer_square_root(42353243534555)